In [ ]:
#libraries
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.stats import ortho_group

In [ ]:
# n is population size and d is the dimension
#def h_gaussian(n, d):
#  coordinates = np.random.normal(0, 1, (n, d))
#  h = coordinates / np.linalg.norm(coordinates)
#  return h

def h_gaussian(n, d):
    coordinates = np.random.normal(0, 1, (n, d))
    h = coordinates / np.linalg.norm(coordinates, axis=1, keepdims=True)
    return h

#def h_RndCoordinates(n, d):
#  rnd_state = np.random.RandomState()
#  coordinates = rnd_state.uniform(-1,1,(n,d))
#  h = coordinates / np.linalg.norm(coordinates)
#  return h

def h_RndCoordinates(n, d):
    rnd_state = np.random.RandomState()
    coordinates = rnd_state.uniform(-1, 1, (n, d))  # Uniform random numbers
    h = coordinates / np.linalg.norm(coordinates, axis=1, keepdims=True)  # Normalize row-wise
    return h

def h_spherical(n, d):
  rnd_state = np.random.RandomState()
  theta = rnd_state.uniform(0, np.pi, size=(n, d-1))
  phi = rnd_state.uniform(0, 2 * np.pi, size=(n, d-1))
  coordinates = np.sin(theta) * np.cos(phi)
  remaining_component = np.prod(np.cos(theta), axis=1)
  h = np.concatenate((coordinates, remaining_component[:, np.newaxis]), axis=1)
  magnitudes = np.linalg.norm(h, axis=1)
  normalized_directions = h / magnitudes[:, np.newaxis]
  return normalized_directions

#def h_orthonormal(n, d): #n<=d
#  if (n> d):
#    raise ValueError("n must be less than or equal to d")
#  x = ortho_group.rvs(d)
#  h = x[:n]/np.linalg.norm(x[:n])
#  return h


def h_orthonormal(n, d):  # n <= d
    if n > d:
        raise ValueError("n must be less than or equal to d")
    x = ortho_group.rvs(d)  # Generate a random orthogonal matrix
    return x[:n]  # Extract and return the first n rows (which are already orthonormal)



In [ ]:
# n is population size and d is the dimension
#def h_gaussian(n, d):
#  coordinates = np.random.normal(0, 1, (n, d))
#  h = coordinates / np.linalg.norm(coordinates)
#  return h

def h_gaussian(n, d):
    coordinates = np.random.normal(0, 1, (n, d))
    h = coordinates / np.linalg.norm(coordinates, axis=1, keepdims=True)
    return h

#def h_RndCoordinates(n, d):
#  rnd_state = np.random.RandomState()
#  coordinates = rnd_state.uniform(-1,1,(n,d))
#  h = coordinates / np.linalg.norm(coordinates)
#  return h

def h_RndCoordinates(n, d):
    rnd_state = np.random.RandomState()
    coordinates = rnd_state.uniform(-1, 1, (n, d))  # Uniform random numbers
    h = coordinates / np.linalg.norm(coordinates, axis=1, keepdims=True)  # Normalize row-wise
    return h

def h_spherical(n, d):
  rnd_state = np.random.RandomState()
  theta = rnd_state.uniform(0, np.pi, size=(n, d-1))
  phi = rnd_state.uniform(0, 2 * np.pi, size=(n, d-1))
  coordinates = np.sin(theta) * np.cos(phi)
  remaining_component = np.prod(np.cos(theta), axis=1)
  h = np.concatenate((coordinates, remaining_component[:, np.newaxis]), axis=1)
  magnitudes = np.linalg.norm(h, axis=1)
  normalized_directions = h / magnitudes[:, np.newaxis]
  return normalized_directions

#def h_orthonormal(n, d): #n<=d
#  if (n> d):
#    raise ValueError("n must be less than or equal to d")
#  x = ortho_group.rvs(d)
#  h = x[:n]/np.linalg.norm(x[:n])
#  return h


def h_orthonormal(n, d):  # n <= d
    if n > d:
        raise ValueError("n must be less than or equal to d")
    x = ortho_group.rvs(d)  # Generate a random orthogonal matrix
    return x[:n]  # Extract and return the first n rows (which are already orthonormal)



In [ ]:
class f_reached_minus_infinity(Exception):
    pass

class target_class_reached_highest_probability(Exception):
    pass

#g(x)
def g(x, f, h, alpha, n):
  print("this is the new g")
  S = 0
  d = x.shape[0]
  h1 = h(n,d).copy()
  try:
    for i in range(n):
      S = S + ((f(x + alpha * h1[i]) - f(x)) / alpha) * h1[i]
    return d/n * S
  except(f_reached_minus_infinity) as e:
    raise
  except(target_class_reached_highest_probability) as e:
    raise

#returns x^k and the values of f(x^i) for i in 1...k
def zero_order(x0, f, alpha, gamma, k, h, n):
  x= x0.copy();
  f_x_i = []
  x_i=[]
  for _ in range(k):
    try:
      x= x - gamma * g(x, f, h, alpha, n)
      x_i.append(x)
      f_x_i.append(f(x))
      print("now we are in zero order:")
      print(x)
      print(f(x))
      print("now we are out")
    except(f_reached_minus_infinity) as e:
      return x, x_i, f_x_i
    except(target_class_reached_highest_probability) as e:
      return x, x_i, f_x_i
  return x, x_i, f_x_i

def graduated_zero_order(x0, f, alpha, gamma, t, h, m, n):
  x = x0.copy()
  f_x_i = []
  for i in range(m):
    #one way for making alpha smaller is mulitplying it with a positive constant smaller than 1, like 0.5
    alpha = alpha/2
    for _ in range(t):
      x= x - gamma * g(x, f, h, alpha, n)
      f_x_i.append(f(x))
  return x, f_x_i

In [ ]:
#We randomly choose the center of the initial search distribution at average distance one from
# the optimum and start with a radial search distribution of unit variance.
def initial_mu_A(x_optimum):
  unit_vector = np.random.randn(len(x_optimum))
  unit_vector /= np.linalg.norm(unit_vector)
  mu_initial = x_optimum + unit_vector
  #Since we want a radial search distribution with unit variance along each dimension, we create an identity matrix for  covariance matrix, C. hence, A
  #such that AA^T = C would also be an identity matrix
  A_initial = np.eye(len(x_optimum))
  return mu_initial, A_initial

#here we give dimention d as an input since the the algorithm generates x0 itself
def xnes (d,f, k, mu, A):
  f_x_k_values = []
  #sigma = np.array([[np.linalg.det(A)]])
  sigma = abs(np.linalg.det(A))**(1/d)
  B = np.array(A/sigma)
# the n mentioned here has nothing to do with the n we used in implemention of directions.
# here parent and offspring population are the same and are both equal to n.
  n =  math.floor(4+3*np.log10(d))
  eta_mu = np.array([[1]])
  eta_sigma = np.array([[0.6 * (3+ np.log10(d))/(d*np.sqrt(d))]])
  eta_B = np.array([[ 0.6 * (3+ np.log10(d))/(d*np.sqrt(d))]])
  for _ in range(k):
    z = h_gaussian(n, d)
    x_tmp = sigma*B@z.T
    x = x_tmp.T + mu
    #finding the index i for which f(x_i) is most fitted(minimum or maimum depending on f).
    #in our case we have to find x_i with the the smallest value for f(x_i)
    values = np.apply_along_axis(f, 1, x)  # Calculate f(x_i) for each vector x_i
    min_index = np.argmin(values)
    f_x_k_values.append(f(x[min_index]))
    u = np.zeros((n,1))
    print(x)
    print("\n")
    for i in range(1, n + 1):
      u[i - 1] = (np.maximum(0, np.log(n / 2 + 1) - np.log(i)) / sum(np.maximum(0, np.log(n / 2 + 1) - np.log(j)) for j in range(1, n + 1))) - (1 / n)
    G_delta = sum(u[i]*z[i] for i in range(n))
    G_M = sum(z[i]*z[i].T - np.identity(d) for i in range(n))
    G_sigma = np.trace(G_M)/d
    G_B = G_M - G_sigma*np.identity(d)
    mu = mu + eta_mu * (sigma*B) @ G_delta
    sigma = sigma*np.exp(eta_sigma/2*G_sigma)
    B = B * np.exp(eta_B/2*G_B)
  return x[min_index], f_x_k_values




In [ ]:
# for all the functions below, x0 is a 1d numpy array

In [ ]:
def ackley(x0):
  a =20
  b = 0.2
  c = 2*math.pi
  d = x0.size
  return -a*math.exp(-b*math.sqrt((x0@x0.T)/d)) - math.exp(np.sum(np.cos(c*x0))/d) +a + math.exp(1)


In [ ]:
def Griewank (x0):
  d = x0.size
  p = 1
  for i in range(1,d+1):
    p = p * math.cos(x0[i-1]/math.sqrt(i))
  return (x0@x0.T)/4000 -p +1

#print (Griewank(np.array([0, 1, 2, 3])))

In [ ]:
def levy(x0):
  d = x0.size
  w = 1 + x0/4 -1/4
  tmp = 0
  for i in range(d-1):
    tmp = tmp + ((w[i]-1)**2)*(1 + 10* (math.sin(math.pi*w[i]+1))**2)
  return (math.sin(math.pi*w[0]))**2 + tmp + ((w[d-1] -1)**2)*(math.sin(2*math.pi*w[d-1])**2)

#print(levy(np.array([2, 2, 2, 2])))

In [ ]:
def Rastrigin(x0):
  d = x0.size
  tmp = 0
  for i in range (d):
    tmp = tmp + x0[i]**2 - 10*math.cos(2*math.pi*x0[i])
  return 10*d + tmp

#print(Rastrigin(np.array([2, 2, 2, 2])))

In [ ]:
def Schwefel(x0):
  d = x0.size
  tmp = 0
  for i in range (d):
    tmp = tmp + x0[i]*math.sin(math.sqrt(np.absolute(x0[i])))
  return 418.9829*d - tmp

#print(Schwefel(np.array([2, 2, 2, 2])))

In [ ]:
#convex
def Rotated_Hyper_Ellipsoid(x0):
  d = x0.size
  tmp = 0
  for i in range (d):
    for j in range(i):
      tmp = tmp + x0[j]**2
  return tmp

#print(Rotated_Hyper_Ellipsoid(np.array([2, 2, 2, 2])))

In [ ]:
#convex
def Sphere(x0):
  return x0@x0.T

In [ ]:
def Sum_Different_Powers(x0):
  d = x0.size
  S = 0
  for i in range(d):
    S = S + (np.absolute(x0[i]))**(i+2)
  return S

In [ ]:
#convex
def Sum_Squares(x0):
  d = x0.size
  S = 0
  for i in range(d):
    S = S + (i+1)*x0[i]*x0[i]
  return S

In [ ]:
def Trid(x0):
  d = x0.size
  return (x0 - 1)@(x0 - 1) - np.delete(x0, 0)@np.delete(x0, d-1)

In [ ]:
def Zakharov(x0):
  d = x0.size
  S = 0
  for i in range(d):
    S = S + 0.5 * (i + 1)* x0[i]
  return np.sum(x0@x0.T + S**2 + S**4)

#print(Zakharov(np.array([2, 2, 2, 2])))

In [ ]:
def Dixon_Price(x0):
  d = x0.size
  S = 0
  for i in range(1,d):
    S = S + (i+1)*(2*x0[i]*x0[i] - x0[i-1])**2
  return (x0[0]-1)**2 + S

#print(Dixon_Price(np.array([0.38334732 -6.92890603])))

In [ ]:
def Rosenbrock(x0):
  d = x0.size
  S = 0
  for i in range(0,d-1):
    S = S + 100*(x0[i+1] - x0[i]**2)**2 + (x0[i]-1)**2
  return S


In [ ]:
#since the i range we are using is 0,..,d-1 and the i range the function is using is 1,..,d, instead of 4i calculate 4(i+1)-1
def Powell(x0):
  d = x0.size
  S = 0
  for i in range(d//4):
    S = S + (x0[4*i] + 10*x0[4*i+1])**2 + 5*(x0[4*i+2] - x0[4*i+3])**2 + (x0[4*i+1] - 2*x0[4*i+2])**4 + 10*(x0[4*i] - x0[4*i+3])**4
  return S



In [ ]:
def Styblinski_Tang(x0):
  d = x0.size
  S = 0
  for i in range(d):
    S = S + x0[i]**4 - 16*(x0[i]**2) +5*x0[i]
  return 0.5*S

In [ ]:
#convex
#def LogSumExp (x0):
#  d = x0.size
#  S = 0
#  for i in range(d):
#    S = S + math.exp(x0[i])

#  return math.log10(S)


In [ ]:
#convex
def Euclidean_Norm(x0):
  return math.sqrt(x0@x0.T)

#print (Euclidean_Norm(np.array([2, 2, 2, 2])))

In [ ]:
#convex
def P_Norm(x0, P=5):
  d = x0.size
  S = 0
  for i in range(d):
    S = S + (abs(x0[i]))**P
  return (np.sum(S))**(1/P)